In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import *
import os
from utils import *

ddir = '../../data/'
dname = 'complete'
nbname = 't5-m_13\''
outputdir="/scratch/userid/" + nbname + '/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
resultsfile = 'results/' + nbname + '.csv'
from sklearn.metrics import classification_report

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1


In [ ]:
df_train = pd.read_csv(ddir + dname + '_train.csv')
df_test = pd.read_csv(ddir + dname + '_test.csv')
df_val = pd.read_csv(ddir + dname + '_val.csv')

In [ ]:
df_train

In [ ]:
def preprocess_df_m_1(df):
    X = []
    y = []
    ids = []
    for index, row in df.iterrows():
        try:
            if not isinstance(row['mislocation'], str) or not isinstance(row['mistype'], str) or row['mistype'] == 'Doubtful type' or row['mislocation'] == 'Doubtful location':
                continue
            tmp = ""
            tmp = "<claim> " + row['claim'] + "\n" if isinstance(row['claim'], str) else "<relation> "
            tmp += "<context> " + row['context'] + "\n" if isinstance(row['context'], str) else "<context> "
            tmp2 = "<source> " + row['Source'] + '\n' if isinstance(row['Source'], str) else "<source> "
            tmp2 += "<relation> " + row['Relation'] + '\n' if isinstance(row['Relation'], str) else "<relation> "
            tmp2 += "<target> " + row['Target'] + '\n' if isinstance(row['Target'], str) else "<target> " 
            tmp2 += "<mismatch> " + row['Mismatch'] + '\n' 
            tmp2 += "<mislocation> " + row['mislocation'] + "\n"
            tmp2 += "<mistype> " + row['mistype']
            X.append(tmp)
            y.append(tmp2)
            ids.append(row['id'])
        except:
            continue
    return X,y, ids

In [ ]:
X_train, y_train, id_train = preprocess_df_m_1(df_train)
X_val, y_val, id_val = preprocess_df_m_1(df_val)
X_test, y_test, id_test = preprocess_df_m_1(df_test)

In [ ]:
len(X_train), len(X_val), len(X_test)

In [ ]:
train_df = pd.DataFrame({'source_text': X_train, 'target_text': y_train, 'id': id_train})
val_df = pd.DataFrame({'source_text': X_val, 'target_text': y_val, 'id': id_val})
test_df = pd.DataFrame({'source_text': X_test, 'target_text': y_test, 'id': id_test})

In [ ]:
train_df

In [ ]:
# load model
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

In [ ]:
# train model
delete_whole_folder(outputdir)
model.train(train_df=train_df,
            eval_df=val_df, 
            source_max_token_len=256, 
            target_max_token_len=256, 
            batch_size=16, 
            max_epochs=4, 
            outputdir = outputdir,
            )
            


In [ ]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in dirs:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result
find('simplet5-epoch-3*', outputdir)[0]

In [ ]:
from tqdm import tqdm
# load the best model

last_epoch_model = outputdir + find('simplet5-epoch-3*', outputdir)[0] # put the name here
model.load_model("t5-small", last_epoch_model, use_gpu=True)
# model.eval()
# for each test data perform prediction
predictions = []
for index, row in tqdm(test_df.iterrows()):
    prediction = model.predict(row['source_text'])[0]
    predictions.append(prediction)


# computer performance
test_df['predicted_label'] = predictions
test_df['original_label'] = test_df['target_text']

In [ ]:
test_df

In [ ]:
def process_output_r(x):
    x_split = x.split(' ')
    # ls = ['',]
    ls = ['', '', '','', '', '']
    itr = None
    for i in x_split:
        if i == 'source>':
            itr = 0
        elif i == 'relation>':
            itr = 1
        elif i == 'target>':
            itr = 2
        elif i == 'mismatch>':
            itr = 3
        elif i == 'mislocation>':
            itr = 4
        elif i == 'mistype>':
            itr = 5
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0], ls[1], ls[2], ls[3], ls[4], ls[5]])


def process_input_r(x):
    x_split = x.split()
    # ls = ['',]
    ls = ['', '', '','', '', '']
    itr = None
    for i in x_split:
        if i == '<source>':
            itr = 0
        elif i == '<relation>':
            itr = 1
        elif i == '<target>':
            itr = 2
        elif i == '<mismatch>':
            itr = 3
        elif i == '<mislocation>':
            itr = 4
        elif i == '<mistype>':
            itr = 5
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0], ls[1], ls[2], ls[3], ls[4], ls[5]])

In [ ]:
test_df[['pSource', 'pRelation', 'pTarget', 'pMismatch', 'pMislocation', 'pMistype']] = test_df['predicted_label'].apply(process_output_r)
test_df[['Source', 'Relation', 'Target', 'Mismatch', 'Mislocation', 'Mistype']] = test_df['original_label'].apply(process_input_r)

In [ ]:
# df_test['predicted_label'] = predictions
# df_test['original_label'] = test_df['target_text']

In [ ]:
test_df

In [ ]:
def exact_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        if orig[i].strip() == predicted[i].strip():
            # print(orig[i], "&", predicted[i])
            match += 1
    return match/len(orig)

def partial_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        orig_split = orig[i].split()
        predicted_split = predicted[i].split()
        if len(orig_split) == 0:
            if len(predicted_split) == 0:
                match += 1
            else:
                match += 0
            continue
        ints = list(set(orig_split) & set(predicted_split))
        match += len(ints)/len(orig_split)
    return match/len(orig)

def IOU(orig, predicted):
    match = 0
    for i in range(len(orig)):
        orig_split = orig[i].split()
        predicted_split = predicted[i].split()
        if len(orig_split) == 0:
            if len(predicted_split) == 0:
                match += 1
            else:
                match += 0
            continue
        ints = list(set(orig_split) & set(predicted_split))
        match += len(ints)/(len(orig_split)+len(predicted_split)-len(ints))
    return match/len(orig)

In [ ]:
exact_match(list(test_df.pSource), list(test_df.Source)), partial_match(list(test_df.pSource), list(test_df.Source)), IOU(list(test_df.pSource), list(test_df.Source))

In [ ]:
exact_match(list(test_df.pRelation), list(test_df.Relation)), partial_match(list(test_df.pRelation), list(test_df.Relation)),  IOU(list(test_df.pRelation), list(test_df.Relation))

In [ ]:
exact_match(list(test_df.pTarget), list(test_df.Target)), partial_match(list(test_df.pTarget), list(test_df.Target)), IOU(list(test_df.pTarget), list(test_df.Target))

In [ ]:
exact_match(list(test_df.Mismatch), list(test_df.pMismatch)), partial_match(list(test_df.Mismatch), list(test_df.pMismatch)), IOU(list(test_df.Mismatch), list(test_df.pMismatch))

In [ ]:
exact_match(list(test_df.Mislocation), list(test_df.pMislocation)), partial_match(list(test_df.Mislocation), list(test_df.pMislocation))

In [ ]:
exact_match(list(test_df.Mistype), list(test_df.pMistype)), partial_match(list(test_df.Mistype), list(test_df.pMistype))

In [ ]:
test_df.to_csv(resultsfile)

In [ ]:
print(classification_report(test_df.Mislocation, test_df.pMislocation))

In [ ]:
print(classification_report(test_df.Mistype, test_df.pMistype))

In [ ]:
join_df = test_df[["id","pSource","pRelation","pTarget","pMismatch","pMislocation","pMistype"]]
res_df = df_test.join(join_df.set_index('id'), on='id')
res_df.to_csv(resultsfile)

In [ ]:
res_df.columns